### Q1: For each of the best seller book mentioned on the first two pages of the following link
https://www.amazon.in/gp/bestsellers/books/
Scrape the following information:
i. Publisher
ii. Language
iii. Paperback
iv. ISBN-10
v. ISBN-13
vi. Weight
vii. Dimensions
viii. First Three Customer Reviews

In [15]:
import requests
from bs4 import BeautifulSoup
import time

In [16]:
def getData(pageno):
    main_url=("http://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_"+str(pageno)+"?ie=UTF8&pg="+str(pageno))
    cnt = itr_cnt = 50
    while(itr_cnt):
        r_main=requests.get(main_url)
        if(r_main.status_code == 200):
            break
        time.sleep(0.1*(cnt-itr_cnt))
        itr_cnt = itr_cnt -1
    if(r_main.status_code != 200):
        print(main_url," link unable to open!\n",book_link)
        return []
    book_no = (pageno-1)*50
    soup=BeautifulSoup(r_main.content)
    all_data=[]
    for d in soup.find_all('div',attrs={'class':'a-section a-spacing-none aok-relative'}):
        book_no = book_no + 1
        name=d.find('span', attrs={'class':'zg-text-center-align'})
        book_name=name.find('img',alt=True).get('alt')
        book_link=d.find('a',attrs={'class':'a-link-normal'})
        book_link = 'http://www.amazon.in/'+ book_link.get('href')
        #print(book_link)
        cnt = itr_cnt = 100
        while(itr_cnt):
            try:
                r_book = requests.get(book_link)
                if(r_book.status_code == 200):
                    break
            except:
                pass
            time.sleep(0.2*(cnt-itr_cnt))
            itr_cnt = itr_cnt -1
        if(r_book.status_code != 200):
            print(book_no," Book link unable to open!\n",book_link)
            continue
        soup_book=BeautifulSoup(r_book.content)
        sub_data=[]
        sub_data.append(book_name)
        book_details=soup_book.find('ul',attrs={'class':'a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list'})
        for data in book_details.find_all('span',attrs={'class':"a-list-item"}):
            f_data = data.text.strip("\n").split("\n\u200f\n:\n\u200e\n\n")
            Publisher = Language = Paperback = ISBN_10 = ISBN_13= Item_Weight = Dimensions = None
            if(f_data[0] == 'Publisher'):
                Publisher = f_data[1]
            if(f_data[0] == 'Language'):
                Language = f_data[1]
            if(f_data[0] == 'Paperback'):
                Paperback = f_data[1]
            if(f_data[0] == 'ISBN-10'):
                ISBN_10 = f_data[1]
            if(f_data[0] == 'ISBN-13'):
                ISBN_13 = f_data[1]
            if(f_data[0] == 'Item Weight'):
                Item_Weight = f_data[1]
            if(f_data[0] == 'Dimensions'):
                Dimensions = f_data[1]
        rev1=rev2=rev3=None
        review_list = soup_book.find('div',attrs={'class':'a-section review-views celwidget'})
        if(review_list is not None):
            rev_count = 0
            for review in review_list.find_all('div',attrs={'class':'a-expander-content reviewText review-text-content a-expander-partial-collapse-content'}):
                review_data = review.find('span')
                if(review_data is None):
                    continue
                rev_count = rev_count + 1
                if(rev_count == 1):
                    rev1 = review_data.text.strip("\n")
                if(rev_count == 2):
                    rev2 = review_data.text.strip("\n")
                if(rev_count == 3):
                    rev3 = review_data.text.strip("\n")
                    break
        sub_data = sub_data+[Publisher,Language,Paperback,ISBN_10,ISBN_13,Item_Weight,Dimensions,rev1,rev2,rev3]
        all_data.append(sub_data)
        print(book_no," processing DONE!")
    return all_data

In [ ]:
import pandas as pd
result=[]
no_pages = 2
for i in range(1,no_pages+1):
    result.append(getData(i))
flatten = lambda l:[item for sublist in l for item in sublist]
df1 = pd.DataFrame(flatten(result),columns=['Book Name','Publisher','Language','Paperback','ISBN-10','ISBN-13','Weight','Dimensions','Review-1','Review-2','Review-3'])
df1.shape

1  processing DONE!


In [12]:
df1

,Book Name,Publisher,Language,Paperback,ISBN-10,ISBN-13,Weight,Dimensions,Review-1,Review-2,Review-3
0,The Psychology of Money,Jaico Publishing House (1 September 2020); Jai...,English,252 pages,9390166268,978-9390166268,220 g,20 x 14 x 4 cm,The expectations from this book were sky hig...,See guys this book is for those who have had...,This is a must read for the youngsters of to...
1,400 Days,Westland (8 October 2021),English,352 pages,1542094089,978-1542094085,230 g,12.9 x 2.54 x 19.81 cm,"Decent story. However, the plot is thin and ...",3star because of same old model cleeches.......,Again a signed copy!! Can't wait to read😍😍
2,Atomic Habits: The life-changing million copy ...,Random House Business Books (30 October 2018),English,288 pages,1847941834,978-1847941831,390 g,15.3 x 2.3 x 23.4 cm,I was aware of many concepts of this book th...,And I absolutely mean it what I said in Head...,"Before starting this book, write down some g..."
3,Ikigai: The Japanese secret to a long and happ...,Random House UK; 1st edition (27 September 2017),English,288 pages,178633089X,978-1786330895,247 g,13.6 x 2.3 x 18.4 cm,The book does a decent job of relating the c...,Just read a back of book and its enough dont...,I personally believe that having a purpose o...
4,My First Library: Boxset of 10 Board Books for...,Wonder House Books; First edition (1 January 2...,English,288 pages,9789387779266,978-9387779266,830 g,20 x 14 x 4 cm,I have well researched many books & conclude...,Delivery was on time packaging was excellent...,"It’s a cute collection, but I wasn’t expexti..."
...,...,...,...,...,...,...,...,...,...,...,...
95,Arihant CBSE Term 1 English Language & Literat...,Arihant Publications; First edition (12 Octobe...,English,199 pages,9325795132,978-9325795136,350 g,20.3 x 25.4 x 4.7 cm,None,None,None
96,General Knowledge 2022,Arihant Publications; Forteen edition (8 March...,English,591 pages,932529558X,978-9325295582,480 g,20 x 14 x 4 cm,Good.. all good important covered..there is ...,Excellent book for all students..it is one o...,Good book
97,Arihant CBSE Term 1 Information Technology (Co...,Arihant Publications; First edition (12 Octobe...,English,108 pages,9325795183,978-9325795181,260 g,20.3 x 25.4 x 4.7 cm,Good,None,None
98,"Xam Idea CBSE MCQs Chapterwise For Term I, Cla...",VK Global Publications Pvt Ltd; 2021st edition...,English,108 pages,9391003974,978-9391003975,1 kg 10 g,26.5 x 20 x 4 cm,Recieved the books in a good condition. Qual...,This books contain all the requirements that...,"Must buy, very helpful for class 10 term 1 b..."


### Q2: Consider the following Link:
https://www.imdb.com/search/title/?release_date=2020&sort=num_votes,desc&page=1
The page contains information about movies released in the year 2020 sorted according to number
of votes. Scrape the following information for the first 1000 movies.
(i) Movie name
(ii) IMDB Rating
(iii) Description
(iv) Votes
(v) Director(s)
(vi) Stars
(vii) Metascore
(viii) Movie Genre
(v) Total Duration 

In [13]:
def get_IMDB_Data(pageno):
    url = ('https://www.imdb.com/search/title/?release_date=2020-01-01,2020-12-31&sort=num_votes,desc&start='+str(pageno)+'&ref_=adv_nxt')
    cnt = itr_cnt = 100
    while(itr_cnt):
        r_main=requests.get(url)
        if(r_main.status_code == 200):
            break
        time.sleep(0.1*(cnt-itr_cnt))
        itr_cnt = itr_cnt -1
    if(r_main.status_code != 200):
        print(url," link unable to open!")
        return []
    soup=BeautifulSoup(r_main.content)
    all_data=[]              
    for data in soup.find_all('div',attrs={'class':'lister-item-content'}):
        Movie_name=Rating_val=Description=Votes_val=Director_names=Stars_name= Metascore_val=MovieGenre_val=TotalDuration=None
        Movie_ref = data.find('h3',attrs={'class':'lister-item-header'})
        Movie_name = Movie_ref.find('a')
        Rating_ref = data.find('div',attrs={'class':'ratings-bar'})
        for dx in data.find_all('p',attrs={'class':'text-muted'}):
            if(dx is not None):
                Description = dx.text.strip("\n")
        Votes_ref = data.find('p',attrs={'class':'sort-num_votes-visible'})
        name_ref = data.find('p',attrs={'class':''})
        Metascore_ref = Rating_ref.find('div',attrs={'class':'inline-block ratings-metascore'})
        data_ref = data.find('p',attrs={'class':'text-muted'})
        MovieGenre_val = data_ref.find('span',attrs={'class':'genre'})
        TotalDuration = data_ref.find('span',attrs={'class':'runtime'})
        if(Movie_name is not None):
            Movie_name = Movie_name.text
        if(Rating_ref is not None):
            Rating_val = Rating_ref.find('strong')
            if(Rating_val is not None):
                Rating_val = Rating_val.text
        if(Votes_ref is not None):
            Votes_val = Votes_ref.find('span',attrs={'name':"nv"})
            if(Votes_val is not None):
                Votes_val = Votes_val.get("data-value")
        if(name_ref is not None):
            if('|' in name_ref.text):
                name_lst = name_ref.text.split('|')
            else:
                name_lst = [name_ref.text]
            if('Director' in name_lst[0]):
                Director_names = name_lst[0].strip('\n').split(':')[1].strip("\n")
            if(len(name_lst)>1):
                Stars_name = name_lst[1].strip('\n').split(':')[1].strip("\n")
            elif('Stars' in name_lst[0]):
                Stars_name = name_lst[0].strip('\n').split(':')[1].strip("\n")
        if(Metascore_ref is not None):
            Metascore_val = Metascore_ref.find('span', attrs={'class':'metascore'})
            if(Metascore_val is not None):
                Metascore_val = Metascore_val.text.strip()
        if(MovieGenre_val is not None):
            MovieGenre_val = MovieGenre_val.text.strip("\n")
        if(TotalDuration is not None):
            TotalDuration = TotalDuration.text
        all_data.append([Movie_name,Rating_val,Description,Votes_val,Director_names,Stars_name,
                        Metascore_val,MovieGenre_val,TotalDuration])
    return all_data
        

In [14]:
interval = 50
start = 1
movie =get_IMDB_Data(start)

In [15]:
import pandas as pd
result=[]
interval = 50
start = 1
end = 1000
for i in range(start,end,interval):
    print(i, " processing...")
    result.append(get_IMDB_Data(i))
flatten = lambda l:[item for sublist in l for item in sublist]
df2 = pd.DataFrame(flatten(result),columns=["Movie_name","Rating_val","Description","Votes_val","Director_names","Stars_name",
                        "Metascore_val","MovieGenre_val","TotalDuration"])
df2.shape

1  processing...
51  processing...
101  processing...
151  processing...
201  processing...
251  processing...
301  processing...
351  processing...
401  processing...
451  processing...
501  processing...
551  processing...
601  processing...
651  processing...
701  processing...
751  processing...
801  processing...
851  processing...
901  processing...
951  processing...


(1000, 9)

In [16]:
df2

,Movie_name,Rating_val,Description,Votes_val,Director_names,Stars_name,Metascore_val,MovieGenre_val,TotalDuration
0,Tenet,7.4,"Armed with only one word, Tenet, and fighting ...",433550,Christopher Nolan,"John David Washington, \nRobert Pattinson, \nE...",69,"Action, Sci-Fi, Thriller",150 min
1,The Queen's Gambit,8.6,"Orphaned at the tender age of nine, prodigious...",362163,None,"Anya Taylor-Joy, \nChloe Pirrie, \nBill Camp, ...",None,Drama,395 min
2,Soul,8.1,"After landing the gig of a lifetime, a New Yor...",283638,"Pete Docter, \nKemp Powers","Jamie Foxx, \nTina Fey, \nGraham Norton, \nRac...",83,"Animation, Adventure, Comedy",100 min
3,Wonder Woman 1984,5.4,Diana must contend with a work colleague and b...,236421,Patty Jenkins,"Gal Gadot, \nChris Pine, \nKristen Wiig, \nPed...",60,"Action, Adventure, Fantasy",151 min
4,Birds of Prey: And the Fantabulous Emancipatio...,6.1,"After splitting with the Joker, Harley Quinn j...",213077,Cathy Yan,"Margot Robbie, \nRosie Perez, \nMary Elizabeth...",60,"Action, Adventure, Comedy",109 min
...,...,...,...,...,...,...,...,...,...
995,Das schaurige Haus,5.4,"Hendrik is sixteen, a big city kid, and vexed ...",3251,Daniel Prochaska,"Leon Orlandianyi, \nBenno Rosskopf, \nJulia Ko...",None,"Horror, Mystery, Thriller",100 min
996,American Wisper,5.2,"In June 2016, a loving family was found shot t...",3243,Russ Emanuel,"Christian Barber, \nNaaji Sky Adzimah, \nLanis...",None,"Crime, Thriller",91 min
997,Veneno,8.7,"A biographical story of La Veneno, transgender...",3243,None,"Lola Rodríguez, \nIsabel Torres, \nPaca la Pir...",None,"Biography, Drama",421 min
998,The Nest,7.1,A teenage girl in Glasgow agrees to carry a ba...,3240,None,"Sophie Rundle, \nKatie Leung, \nMartin Compsto...",None,Thriller,60 min
